In [ ]:
import ast
import os

import aesara.tensor as at
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
from sklearn.preprocessing import LabelEncoder

from draft_optimizer.src.utils import DATA_DIR

In [ ]:
def trace_helper(trace, var_names):
    display(az.summary(trace, var_names=var_names, kind="diagnostics"))
    az.plot_trace(trace, var_names=var_names, compact=False)
    plt.show()

In [ ]:
# Load data
league_id = 88497130
years = [2019, 2020, 2021, 2022]
teams_raw, schedule_raw, players_raw = [], [], []
for year in years:
    # Load data
    league_dir = os.path.join(DATA_DIR, f"espn_{league_id}", str(year))
    teams_raw_year = pd.read_csv(os.path.join(league_dir, "pro_teams.csv"))
    schedule_raw_year = pd.read_csv(os.path.join(league_dir, "pro_schedule.csv"))
    players_raw_year = pd.read_csv(os.path.join(league_dir, "pro_players.csv"))

    # Add year
    teams_raw_year["year"] = year
    schedule_raw_year["year"] = year
    players_raw_year["year"] = year

    # Append
    teams_raw.append(teams_raw_year)
    schedule_raw.append(schedule_raw_year)
    players_raw.append(players_raw_year)

# Concat
teams_raw = pd.concat(teams_raw, axis=0, ignore_index=True)
schedule_raw = pd.concat(schedule_raw, axis=0, ignore_index=True)
players_raw = pd.concat(players_raw, axis=0, ignore_index=True)

# Export for PyMC help
teams_raw.to_csv(os.path.join(DATA_DIR, f"espn_{league_id}", "teams_raw.csv"), index=False)
schedule_raw.to_csv(os.path.join(DATA_DIR, f"espn_{league_id}", "schedule_raw.csv"), index=False)
players_raw.to_csv(os.path.join(DATA_DIR, f"espn_{league_id}", "players_raw.csv"), index=False)

In [ ]:
# Get maps using latest abbrev/name; ex: Commanders D/ST instead of Washington D/ST
teams_map = teams_raw.groupby("id")["abbrev"].last()
teams_map = pd.Series(teams_map.index.values, index=teams_map)
players_map = players_raw.groupby("id")["name"].last()
players_map = pd.Series(players_map.index.values, index=players_map)

In [ ]:
# Prepare weekly data
players_df = players_raw.copy()
players_df["team_id"] = players_df["pro_team"].map(teams_map)
players_df = players_df[["id", "position", "year", "team_id"]].rename({"id": "player_id"}, axis=1)
home_data = schedule_raw[["home_id", "away_id", "year", "week"]].merge(
    players_df, left_on=["year", "home_id"], right_on=["year", "team_id"]
)
home_data["home"] = True
home_data["opponent_id"] = home_data["away_id"]
away_data = schedule_raw[["home_id", "away_id", "year", "week"]].merge(
    players_df, left_on=["year", "away_id"], right_on=["year", "team_id"]
)
away_data["home"] = False
away_data["opponent_id"] = away_data["home_id"]
weekly_data = (
    pd.concat([home_data, away_data], axis=0).drop(["team_id", "home_id", "away_id"], axis=1).dropna(how="any")
)
weekly_data = weekly_data.sort_values(["year", "week", "player_id"])

# Get years
years = sorted(weekly_data["year"].unique())

print(weekly_data.shape)
weekly_data.head()

In [ ]:
# Get historic weekly data
historic_weekly_points = players_raw.set_index(["year", "id"])["weekly_points"]
historic_weekly_points = historic_weekly_points.loc[
    historic_weekly_points.index.get_level_values(0) != years[-1]
]  # exclude latest year, i.e. the one we're making projections for
historic_weekly_points = historic_weekly_points.apply(lambda v: ast.literal_eval(v))  # str -> dict
historic_weekly_points = historic_weekly_points.apply(lambda v: pd.Series(v, dtype=float))
historic_weekly_points.columns.name = "week"
historic_weekly_points = historic_weekly_points.stack()  # drops NaN rows, i.e. where a player didn't see the field
historic_weekly_points = historic_weekly_points.reset_index().rename({"id": "player_id", 0: "points"}, axis=1)
historic_weekly_data = weekly_data.merge(
    historic_weekly_points, left_on=["year", "week", "player_id"], right_on=["year", "week", "player_id"], how="right"
)

print(historic_weekly_data.shape)
historic_weekly_data.head()

In [ ]:
# Get future points projection
proj_points = players_raw.loc[players_raw["year"] == years[-1]].groupby("id")["proj_points"].first()
proj_points = proj_points.loc[proj_points > 0]  # exclude players who aren't expected to see the field

# Get future weekly data (i.e. the year we're making projects for)
future_weekly_data = weekly_data.loc[weekly_data["year"] == years[-1]]
future_weekly_data = future_weekly_data.loc[future_weekly_data["player_id"].isin(proj_points.index)]

print(proj_points.shape)
display(proj_points.head())
print(future_weekly_data.shape)
future_weekly_data.head()

# One Position, Just Historic
* TODO: weekly effect within each year?
* Once this works well, include the projected points as a second likelihood

In [ ]:
# Subset data
pos = "QB"
historic = historic_weekly_data.loc[historic_weekly_data["position"] == pos]
future = future_weekly_data.loc[future_weekly_data["position"] == pos]
players = sorted(set(historic["player_id"].unique()) | set(future["player_id"].unique()))
pos_proj_points = proj_points.loc[future["player_id"]].sort_index()

# Build coords
teams = sorted(
    set(historic["opponent_id"].unique()) | set(future["opponent_id"].unique()) | {np.nan}
)  # include NaN for bye weeks
coords = {"teams": teams, "players": players, "years": years}

# Fit encoders
teams_encoder = LabelEncoder().fit(teams)
players_encoder = LabelEncoder().fit(players)
years_encoder = LabelEncoder().fit(years)

# Prepare historic data
historic_opp_idx = teams_encoder.transform(historic["opponent_id"])
historic_player_idx = players_encoder.transform(historic["player_id"])
historic_year_idx = years_encoder.transform(historic["year"])
historic_home_vals = historic["home"].astype(float).values
historic_points_vals = historic["points"].values

In [ ]:
with pm.Model(coords=coords) as model:
    # Opponent-specific parameters (non-centered format for better sampling)
    opp_std = pm.HalfNormal("opp_std", sigma=7)
    # opp_mu = pm.Normal("opp_mu", mu=0, sigma=7)  # indistinguishable with `player_mu`, so removing this parameter
    # Non-GRW implementation
    # opp_offset = pm.Normal("opp_offset", mu=0, sigma=7, dims=["years", "teams"])
    # opp = pm.Deterministic("opp", opp_std * opp_offset, dims=["years", "teams"])  # opp_mu
    # GRW implementation
    opp_offset = pm.Normal("opp_offset", mu=0, sigma=7, dims=["years", "teams"])
    yearly_opp = pm.Deterministic("yearly_opp", opp_std * opp_offset, dims=["years", "teams"])  # opp_mu
    opp_zeros = at.shape_padleft(at.zeros_like(yearly_opp[0, :]))
    opp_cumsum = yearly_opp.cumsum(axis=0)[:-1]
    opp = pm.Deterministic("opp", yearly_opp - at.concatenate([opp_zeros, opp_cumsum], axis=0), dims=["years", "teams"])

    # Player-specific parameters (non-centered format for better sampling)
    player_std = pm.HalfNormal("player_std", sigma=7)
    player_mu = pm.Normal("player_mu", mu=0, sigma=7)  # indistinguishable with `opp_mu`, so pnly using this parameter
    # Non-GRW implementation
    # player_offset = pm.Normal("player_offset", mu=0, sigma=7, dims=["years", "players"])
    # player = pm.Deterministic("player", player_mu + player_std * player_offset, dims=["years", "players"])
    # GRW implementation
    player_offset = pm.Normal("player_offset", mu=0, sigma=7, dims=["years", "players"])
    yearly_player = pm.Deterministic("yearly_player", player_mu + player_std * player_offset, dims=["years", "players"])
    players_zeros = at.shape_padleft(at.zeros_like(yearly_player[0, :]))
    players_cumsum = yearly_player.cumsum(axis=0)[:-1]
    player = pm.Deterministic(
        "player", yearly_player - at.concatenate([players_zeros, players_cumsum], axis=0), dims=["years", "players"]
    )

    # Home-field advantage
    # Non-GRW implementation
    # home = pm.Normal("home", mu=0, sigma=1, dims="years")
    # GRW implementation
    yearly_home = pm.Normal("yearly_home", mu=0, sigma=1, dims="years")
    home = pm.Deterministic("home", yearly_home - at.concatenate([[0.0], yearly_home.cumsum()[:-1]]), dims="years")
    historic_home = home[historic_year_idx] * historic_home_vals

    # Historic weekly points projection: RV centered on player traits, the opponent, and home-field advantage
    historic_points_mu = (
        player[historic_year_idx, historic_player_idx] + opp[historic_year_idx, historic_opp_idx] + historic_home
    )

    # Likelihood based on historic points projects
    points_std = pm.HalfNormal("points_std", sigma=7)
    historic_points = pm.Normal(
        "historic_points", mu=historic_points_mu, sigma=points_std, observed=historic_points_vals
    )

display(pm.model_to_graphviz(model))
model.point_logps()

In [ ]:
# Sample model
with model:
    trace = pm.sample(draws=1000, tune=2000, init="jitter+adapt_diag_grad")

In [ ]:
# Check opponent-specific parameters
trace_helper(trace, ["opp_std", "opp_offset", "opp"])  # "opp_mu"

In [ ]:
# Check player-specific parameters
trace_helper(trace, ["player_std", "player_mu", "player_offset", "player"])

In [ ]:
# Check miscellaneous parameters
trace_helper(trace, ["home"])

In [ ]:
az.plot_pair(trace, var_names=["opp_std", "player_std", "player_mu"], coords=coords, divergences=True)
# "opp_mu"

In [ ]:
# Plot posterior historic points vs. actual historic points
with model:
    posterior = pm.sample_posterior_predictive(trace)
posterior_historic_points = posterior.posterior_predictive["historic_points"].mean(axis=0).mean(axis=0)
fig, ax = plt.subplots()
ax.scatter(historic_points_vals, posterior_historic_points, alpha=0.15, color="blue")
points_min = np.min([historic_points_vals.min(), posterior_historic_points.values.min()])
points_max = np.max([historic_points_vals.max(), posterior_historic_points.values.max()])
diag = np.linspace(points_min, points_max, 25)
ax.plot(diag, diag, alpha=0.5, ls="--", color="orange")
ax.set_xlabel("Actual")
ax.set_ylabel("Predicted")
plt.show()